In [18]:
import pandas as pd
import os
import seaborn as sns
import shutil

os.environ["CRYPTOGRAPHY_OPENSSL_NO_LEGACY"] = "yes"

# ? Se usa para generar gráficos
import matplotlib.pyplot as plt

# ? Se usa para determina la fecha de ejecución del proceso dentro del export de los resultados estadísticos
from datetime import datetime

# ? Conexión a BD Postgres. Se usa dado que pandas no acepta sino este paquete para la conexión.
from sqlalchemy import create_engine, text

from datetime  import datetime as dt

from arcgis.features import GeoAccessor, GeoSeriesAccessor
import arcpy

from pathlib import Path

# ** Librerías propias

In [19]:
RUTA_CONSOLIDACION_SALDOS = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Id_FisicoJuridica\2_2_9_Seguimiento_EConservacion\zInsumos\20240719.csv"
ruta_bd_terrenos_w_gis = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E1_Alistamiento_Diagnostico\3_Disposicion\1.BD_Consolidada\BD_Consolidada_Lote4.gdb\Analitica_UT_Consolidada\TERRENO_UNIFICADO_w_GIS"

estandarizacion_saldos_conservacion = ['MUNICIPIO',
    'TIPO',
    'RADICADO IGAC',
    'FECHA RAD IGAC',
    'RADICADO BCGS',
    'REFERENCIA CATASTRAL',
    'MUTACIÓN',
    'FECHA DE VISITA',
    'INFORME DE VISITA',
    'SHF',
    'INFORME TÉCNICO UT',
    'ESTADO DE LA REVISIÓN',
    'FECHA REVISIÓN CONSERVACIÓN',
    'ESTADO DE REVISIÓN',
    'ENVIADO A IGAC',
    'FECHA DE ENVÍO'
    ]

estandarizacion_terrenos_saldosConservacion = [
    'codigo',
    'MUNICIPIO',
    'MUTACIÓN',
    'INFORME DE VISITA',
    'SHF',
    'INFORME TÉCNICO UT',
    'ESTADO DE REVISIÓN',
    'ENVIADO A IGAC',
    'SHAPE'
    ]

In [20]:
df_saldos_conservacion = pd.read_csv(RUTA_CONSOLIDACION_SALDOS)

In [21]:
df_saldos_conservacion = df_saldos_conservacion[estandarizacion_saldos_conservacion]

In [22]:
df_terrenos = pd.DataFrame.spatial.from_featureclass(ruta_bd_terrenos_w_gis)

estandarizacion_terrenos  = ['codigo','area_ha_cmt12','SHAPE']

In [23]:
df_terrenos_saldosConservacion = pd.merge(left=df_terrenos,
    right=df_saldos_conservacion,
    left_on='codigo',
    right_on='REFERENCIA CATASTRAL',
    how='inner')

df_terrenos_saldosConservacion = df_terrenos_saldosConservacion[estandarizacion_terrenos_saldosConservacion]

df_terrenos_saldosConservacion

,codigo,MUNICIPIO,MUTACIÓN,INFORME DE VISITA,SHF,INFORME TÉCNICO UT,ESTADO DE REVISIÓN,ENVIADO A IGAC,SHAPE
0,134420000000000030445000000000,MARIA LA BAJA,MUTACIÓN QUINTA NUEVO,SI,SI,SI,Completado,SI,"{""rings"": [[[4739659.7288, 2653287.8803000003]..."
1,134420000000000030129000000000,MARIA LA BAJA,MUTACIÓN QUINTA NUEVO,SI,SI,SI,Completado,NO,"{""rings"": [[[4736507.3453, 2658677.9493000004]..."
2,134420000000000040695000000000,MARIA LA BAJA,RECTIFICACIONES,SI,SI,SI,Completado,SI,"{""rings"": [[[4740546.3004, 2648249.6193000004]..."
3,134420000000000040720000000000,MARIA LA BAJA,RECTIFICACIONES,SI,NaN,NaN,Completado,NO,"{""rings"": [[[4741467.9717, 2650407.9857], [474..."
4,134420000000000040933000000000,MARIA LA BAJA,MUTACIÓN SEGUNDA DESENGLOBE,SI,SI,SI,Completado,SI,"{""rings"": [[[4743987.0485, 2647234.6100999992]..."
...,...,...,...,...,...,...,...,...,...
145,080780100000003940006000000000,BARANOA,MUTACIÓN SEGUNDA DESENGLOBE,SI,SI,SI,Completado,NO,"{""rings"": [[[4790372.0393, 2750865.986300001],..."
146,080780100000000500011000000000,BARANOA,RECTIFICACIONES,NaN,NaN,NaN,Devuelto,NO,"{""rings"": [[[4790526.2838, 2751902.1788], [479..."
147,080780100000001760009000000000,BARANOA,MUTACIÓN SEGUNDA DESENGLOBE,SI,SI,SI,Pendiente,NO,"{""rings"": [[[4790335.4706, 2751264.696799999],..."
148,080780100000007600001000000000,BARANOA,REVISIÓN DE AVALÚO,SI,NaN,SI,Completado,NO,"{""rings"": [[[4791521.4189, 2750750.1522000004]..."


In [24]:
RUTA_BDLOCAL = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Id_FisicoJuridica\2_2_9_Seguimiento_EConservacion\Base_Datos\2_2_9_SegEConservacion.gdb"
NOMBRE_FEATURE = 'Seguimiento_EConservacion'

df_terrenos_saldosConservacion.spatial.to_featureclass(location=os.path.join(RUTA_BDLOCAL, NOMBRE_FEATURE))
print(f"Se genera el featureclass {NOMBRE_FEATURE}")

Se genera el featureclass Seguimiento_EConservacion
